# Plot groups' analysis
## Set parameters

In [ ]:
# ####################################### GENERAL OPTIONS #######################################
#experiment_root = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/data/experiment/"
experiment_root = "./data/experiments/soumnya/experiment/"
#proof_root = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/data/proof/"
proof_root = "./data/experiments/soumnya/proof/"
#plots_output_path = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/results/"
plots_output_path = "./plots/soumnya/"

mode = "sum"
marker = "CFos"
plot_height = 900
plot_width = 900
plot_max_x = 4000
plot_max_y = 4000
show_plot = True
save_plot = False
saved_plot_extension = ".html"                      # '.html' for interactive plot
                                                    # '.svg' for vectorized image
                                                    # '.png'/'.jpg'/... for rasterized image

## Script's code
run all cell below

In [ ]:
import BraiAn
import os

experiment_input_path = os.path.join(experiment_root, "BraiAn_output")
proof_input_path = os.path.join(proof_root, "BraiAn_output")

In [ ]:
animals_sums = []

for file in os.listdir(proof_input_path):
    if os.path.isfile(os.path.join(proof_input_path, file)) and file.endswith(f"_{mode}.csv"):
        animal = file[:-len(f"_{mode}.csv")]
        experiment_animal = BraiAn.AnimalBrain.from_csv(animal, experiment_input_path, mode=mode)
        proof_animal = BraiAn.AnimalBrain.from_csv(animal, proof_input_path, mode=mode)
        animals_sums.append((BraiAn.AnimalBrain.merge_hemispheres(experiment_animal), BraiAn.AnimalBrain.merge_hemispheres(proof_animal)))

In [ ]:
animals = []
regions = []
experiment_density = []
proof_density = []
for experiment_animal, proof_animal in animals_sums:
    proof_animal_densities = proof_animal.data[marker] / proof_animal.data["area"]
    experiment_animal_densities = experiment_animal.data[marker] / experiment_animal.data["area"] 
    for brain_region in proof_animal_densities.index:
        if brain_region in experiment_animal_densities.index:
            animals.append(experiment_animal.name)
            regions.append(brain_region)
            experiment_density.append(experiment_animal_densities[brain_region])
            proof_density.append(proof_animal_densities[brain_region])


In [ ]:
import plotly.graph_objects as go

hover_text = [f"<b>animal</b>: {animal}<br><b>region</b>: {region}<br><b>experiment</b>: {experiment}<br><b>proof</b>: {proof}" for animal, region, experiment, proof in zip(animals, regions, experiment_density, proof_density)]

fig = go.Figure([
    go.Scatter(
        x=experiment_density,
        y=proof_density,
        text=hover_text,
        hoverinfo='text',
        mode="markers"
    )]
)

fig.update_layout(
    xaxis=dict(
        title="experiment",
        range=[0, plot_max_x]
    ),
    yaxis=dict(
        title="proof",
        range=[0, plot_max_y]
    ),
    height=plot_height,
    width=plot_width
)

if save_plot:
    plot_filename = f"correlation_experiment_vs_proof{saved_plot_extension}".lower()
    plot_filepath = os.path.join(plots_output_path, plot_filename)
    match saved_plot_extension.lower():
        case ".html":
            fig.write_html(plot_filepath)
        case _:
            fig.write_image(plot_filepath)

if show_plot:
    fig.show()

In [ ]:
from scipy.stats import pearsonr
r,p = pearsonr(experiment_density, proof_density)
r,p